<a href="https://colab.research.google.com/github/saraarizzi/chess-analysis/blob/main/AudioClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os
from google.colab import drive
from shutil import copyfile
import shutil
from time import time
import tarfile
import IPython.display as ipd
from matplotlib import pyplot as plt
from scipy.io import wavfile as wav
from scipy.fft import rfft, fft, irfft, ifft, fftfreq
from scipy.signal import fftconvolve
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

import librosa
import librosa.display
from sklearn.preprocessing import scale

In [2]:
drive.mount('/content/drive')
tar_path = '/content/drive/MyDrive/recordings.tar'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with tarfile.open(tar_path, 'r') as tar_ref:
    tar_ref.extractall('/content/drive/MyDrive/audio_unzipped')

In [4]:
# shutil.rmtree('/content/drive/MyDrive/audio_unzipped')

Ci occupiamo ora di fare **classfiicazione**, ovvero a partire dal dataset di soggetti diversi che pronunciano le cifre, cercare di classificare la voce associandola alla persona corretta.
Oppure, possiamo fare in modo di classificare per la corretta cifra pronunciata.

Per i task di classificazione, dunque di ML, a differenza degli approcci di deep learnin come le reti neurali, non possiamo passare in input il dato grezzo, ma bisogna processarne ed estrarne le **features**, da usare per il modello di classificazione.

Andremo pertanto a calcolare una serie di indicatori (media della frequenza, min max etc.).

Come sappiamo, avremo bisogno di una fase di addestramento, e le corrispondenti classi (es. possiamo usare come classe la cifra associata alla pronuncia).

# Dataloader

Per prima cosa inizializziamo il **dataloader**, ossia una classe python, o una funzione, il cui scopo è caricare e fornire al sistema i dati necessari all'addestramento, preparati secondo quanto indicheremo noi.

In [9]:
# Placecholder for feature extractor
# Tuttavia, se usassimo il segnale grezzo per le funzioni successive,
# il codice non funzionerà, poichè i segnali hanno tutti len() diverse,
# pertanto dobbiamo rincodurli tutti alla stessa len()
def identity(input):
    return input

# Estraiamo i primi 100 elementi delle tracce audio. Se len()<100, allora
# appplichiamo un padding di zeri alla fine dell'array, per renderlo len()==100
def crop(input):
    if len(input) < 100:
        output = np.pad(input, (0, 100 - len(input)))
    else:
      output = input[:100]
    return output

# funzione del Data loader vera e propria
# come parametri è passato l'input grezzo (funzione identità),
def load_data(feature_extractor=identity, normalize=False):

    labels = [] # inizializza la lista delle etichette
    features = [] # inizializza le features

    directory = '/content/drive/MyDrive/audio_unzipped/recordings/'
    for f in sorted(os.listdir(directory)): # ciclo che legge i file .wav...
        if f.endswith('.wav'):
            # Load file and compute the requested features
            _, signal = wav.read(directory + f)   # ... li importa, senza salvare il sample_rate poichè è sempre uguale
            cur_features = feature_extractor(signal) # il segnale letto alla riga precedente è passato come parametro per
            features.append(cur_features) # la funzione di feature_extraction, ed esse saranno aggiunte alla lista associata

            # Classes
            label = f.split('_')[0]  # isoliamo le classi
            labels.append(label) # e le appendiamo alla lista corrispondente

    # X: features, y: labels
    # Prepariamo il train e test set, con prop 90-10
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=1)
  # random_state è un seed per la selezione casuale dei dati, che quindi diventa riproducibile
    if normalize:
        eps = 0.001
        X_train = np.array(X_train)
        X_train_mean = X_train.mean(axis=0)
        X_train_std = X_train.std(axis=0)
        X_train = (X_train - X_train_mean + eps)/(X_train_std + eps)
        X_train = [row for row in X_train]

        X_test = [row for row in (np.array(X_test) - X_train_mean + eps)/(X_train_std + eps)]

    return X_train, X_test, y_train, y_test

Usiamo adesso il **data_loader**.
Passeremo come parametro la funzione 'crop' al feature_extractor

In [10]:
X_train, X_test, y_train, y_test = load_data(feature_extractor=crop, normalize=True)

In [17]:
# Verifichiamo la grandezza di X_train ossia quante tracce audio
# sono entrate nel training
len(X_train)

1350

Verifichiamo che al funzione 'crop' abbia funzionato.
Mi aspetto che tutti gli elementi di X_train abbiano stessa shape, ovvero [100,]

In [15]:
# Ne prendiamo alcuni a caso
print(X_train[0].shape)
print(X_train[120].shape)
print(X_train[490].shape)
print(X_train[760].shape)
print(X_train[1200].shape)

# Abbiamo verificato che tutte le tracce hanno stessa shape di campioni

(100,)
(100,)
(100,)
(100,)
(100,)


### SVM application